In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential , load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import numpy as np

In [7]:
train_dir = r"D:\college\archive\train"
valid_dir = r"D:\college\archive\valid"
test_dir = r"D:\college\archive\test"

In [9]:
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(64, 64), batch_size=32, class_mode='binary')
valid_generator = valid_datagen.flow_from_directory(valid_dir, target_size=(64, 64), batch_size=32, class_mode='binary')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(64, 64), batch_size=32, class_mode='binary')

Found 30250 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.


In [11]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64,(3,3),activation = 'relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128,activation='relu'),
    Dropout(0.5),
    Dense(1,activation='sigmoid')
])

C:\Users\divya\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [15]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [17]:
history = model.fit(train_generator,epochs=4,validation_data=valid_generator,verbose = 1)

C:\Users\divya\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/4
946/946 ━━━━━━━━━━━━━━━━━━━━ 129s 134ms/step - accuracy: 0.8613 - loss: 0.3003 - val_accuracy: 0.9190 - val_loss: 0.1980
Epoch 2/4
946/946 ━━━━━━━━━━━━━━━━━━━━ 97s 102ms/step - accuracy: 0.9309 - loss: 0.1837 - val_accuracy: 0.9490 - val_loss: 0.1416
Epoch 3/4
946/946 ━━━━━━━━━━━━━━━━━━━━ 97s 103ms/step - accuracy: 0.9405 - loss: 0.1627 - val_accuracy: 0.9557 - val_loss: 0.1151
Epoch 4/4
946/946 ━━━━━━━━━━━━━━━━━━━━ 97s 103ms/step - accuracy: 0.9460 - loss: 0.1475 - val_accuracy: 0.9557 - val_loss: 0.1200


In [19]:
loss,accuracy = model.evaluate(test_generator, verbose=1)
print('Test accuracy:', accuracy)

197/197 ━━━━━━━━━━━━━━━━━━━━ 19s 98ms/step - accuracy: 0.9639 - loss: 0.0966
Test accuracy: 0.9626984000205994


In [21]:
model.save("ffd_model.h5")

In [23]:
model = load_model("ffd_model.h5")

In [25]:
# Function to load and predict an image
def predict_image():
    # Open file dialog to select an image
    file_path = filedialog.askopenfilename()
    if file_path:
        # Display the image in the GUI
        img = Image.open(file_path)
        img = img.resize((200, 200))
        img = ImageTk.PhotoImage(img)    #convert image for tk
        image_label.configure(image=img) #update the image in GUI
        image_label.image = img

        # Preprocess the image for the model
        img_for_model = Image.open(file_path).resize((64, 64))
        img_array = np.array(img_for_model) / 255.0  # Rescale like during training
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

        # Make a prediction
        prediction = model.predict(img_array)[0][0] #extracts the scalar prediction value
        result = "Wildfire" if prediction > 0.5 else "No Wildfire"
        result_label.config(text="Prediction: " + result)

# Setting up the GUI window
root = tk.Tk()
root.title("Forest Fire Detection")
root.geometry("400x400")

# Add widgets
btn = tk.Button(root, text="Upload Image", command=predict_image) #button triggers the predict_image() function when clicked
btn.pack(pady=20)

#Placeholder for displaying the selected image
image_label = tk.Label(root)
image_label.pack()

#Label to display the prediction result
result_label = tk.Label(root, text="Prediction: ", font=("Helvetica", 16))
result_label.pack(pady=20)

#Starts the Tkinter event loop, keeping the GUI active until manually closed
root.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
